In [58]:
import pandas as pd
import datetime as datetime
import time
import pycurl, certifi, json
from pandas.io.json import json_normalize
from io import BytesIO

In [59]:
df = pd.read_csv('311cleaned.csv')

In [60]:
df.head()

,REQUEST_ID,CREATED_ON,REQUEST_TYPE,REQUEST_ORIGIN,STATUS,DEPARTMENT,NEIGHBORHOOD,COUNCIL_DISTRICT,WARD,TRACT,PUBLIC_WORKS_DIVISION,PLI_DIVISION,POLICE_ZONE,FIRE_ZONE,X,Y,GEO_ACCURACY
0,205290,2017-12-31T23:17:00,Snow/Ice removal,Website,1,DPW - Street Maintenance,Marshall-Shadeland,1.0,27.0,4.200327e+10,1.0,27.0,1.0,1-14,-80.034743,40.472661,EXACT
1,205289,2017-12-31T22:01:00,Snow/Ice removal,Report2Gov iOS,1,DPW - Street Maintenance,Stanton Heights,9.0,10.0,4.200310e+10,2.0,10.0,5.0,3-7,-79.936661,40.480244,EXACT
2,205288,2017-12-31T21:55:00,Snow/Ice removal,Website,1,DPW - Street Maintenance,New Homestead,5.0,31.0,4.200331e+10,3.0,31.0,4.0,4-20,-79.927033,40.389386,EXACT
3,205287,2017-12-31T18:23:00,Missed Pick Up,Website,1,DPW - Refuse,Mount Washington,2.0,19.0,4.200319e+10,5.0,19.0,3.0,4-27,-80.014213,40.426213,APPROXIMATE
4,205286,2017-12-31T17:56:00,Snow/Ice removal,Report2Gov iOS,1,DPW - Street Maintenance,Stanton Heights,9.0,10.0,4.200310e+10,2.0,10.0,5.0,3-7,-79.937035,40.480705,EXACT


In [61]:
df.describe()

,REQUEST_ID,STATUS,COUNCIL_DISTRICT,WARD,TRACT,PUBLIC_WORKS_DIVISION,PLI_DIVISION,POLICE_ZONE,X,Y
count,79852.000000,79852.000000,76069.000000,75965.000000,7.612800e+04,75966.000000,75965.000000,75922.000000,76784.000000,76784.000000
mean,164666.873704,1.176514,4.882370,16.369157,4.200323e+10,3.029619,16.369157,3.550815,-79.289558,40.095406
std,23625.934393,0.703213,2.571621,8.025851,1.887250e+05,1.461813,8.025851,1.607305,7.360487,3.722107
min,123268.000000,0.000000,1.000000,0.000000,4.200301e+10,0.000000,0.000000,1.000000,-80.088105,0.000000
25%,144463.750000,1.000000,3.000000,10.000000,4.200311e+10,2.000000,10.000000,2.000000,-80.007287,40.421424
50%,164863.500000,1.000000,5.000000,16.000000,4.200318e+10,3.000000,16.000000,4.000000,-79.977465,40.444844
75%,185093.250000,1.000000,7.000000,22.000000,4.200328e+10,5.000000,22.000000,5.000000,-79.934367,40.461995
max,205290.000000,3.000000,9.000000,32.000000,4.200398e+10,6.000000,32.000000,6.000000,0.000000,40.526962


In [62]:
# https://darksky.net/dev/docs#time-machine-request
dsKey = "9aff7e03ca4b02e487cc0deac0dcbcab/";
dsUrl = "https://api.darksky.net/forecast/";
exclude = "?exclude=minutely,hourly,flags";
# dsAPI =  dsUrl + dsKey + dsLat + ',' + dsLon + ',' + Time + exclude

In [ ]:
dfwx = pd.DataFrame()  #new dataframe to store results
for index, row in df.iterrows():
    if index > 300:  break #limit api calls 10000/$1
    date = datetime.datetime.strptime(row['CREATED_ON'], '%Y-%m-%dT%H:%M:%S') 
    date -= datetime.timedelta(seconds=14400)
    timestamp = int(date.timestamp())
    
    #skip rows with no time/location data, write api calls to file for troubleshooting
    if not (timestamp and row['X'] and row['Y']):
        continue
    else:
        dsAPI =  dsUrl + dsKey + str(row['Y']) + ',' + str(row['X']) + ',' + str(timestamp) + exclude
        with open('apicalls.txt', 'a+') as f:
            f.write(str(index) + ':\t' + dsAPI + "\n")    
        
    buffer = BytesIO()
    c = pycurl.Curl()
    c.setopt(c.URL, dsAPI)
    c.setopt(c.VERBOSE, True)
    c.setopt(c.CAINFO, certifi.where())
    c.setopt(c.WRITEDATA, buffer)
    c.perform()
    c.close()     
    wx = json.loads(buffer.getvalue().decode('iso-8859-1'))
    buffer.close()
    
    #should have been able to use json_normalize direct from that path. didn't work. Did it below.
    #https://www.kaggle.com/jboysen/quick-tutorial-flatten-nested-json-in-pandas
#     wxd = json_normalize(wx, 
#                          record_path=[['daily', 'data']], 
#                          meta=['latitude', 'longitude'], 
#                          record_prefix='d_',   #this worked here but not in the json_normalize below
#                          errors='ignore')
    
    if 'daily' in wx:
        wxd = wx["daily"]["data"] 
        #wxd = json_normalize(wxd, record_path=None, record_prefix="daily")
        wxd = json_normalize(wxd) #flattens json into series
        wxd = wxd.add_prefix('daily.')
    else: 
        wxd = pd.Series()
        
    #add precipType field because it's not there if no precip and ruins frame alignment
    if 'currently' in wx:
        wxc = wx['currently']
        if 'precipType' not in wxc:
            wxc['precipType'] = ''
        wxc = json_normalize(wxc)
        wxc = wxc.add_prefix('current.')
    else: 
        wxc = pd.Series()
        
    new_row = pd.concat([pd.Series(row['REQUEST_ID'], name='REQUEST_ID'), wxc, wxd], axis=1, ignore_index=False)
    #print (index, dsAPI, wxd, wxc, new_row)
    print(index)
    dfwx = pd.concat([dfwx, new_row], axis=0, sort=False, ignore_index=False)

In [67]:
dfwx.describe()

,REQUEST_ID,current.apparentTemperature,current.cloudCover,current.dewPoint,current.humidity,current.precipIntensity,current.precipProbability,current.pressure,current.temperature,current.time,...,daily.time,daily.uvIndex,daily.uvIndexTime,daily.visibility,daily.windBearing,daily.windGust,daily.windGustTime,daily.windSpeed,0,1
count,297.000000,290.000000,290.000000,290.000000,290.000000,290.000000,290.000000,290.000000,290.000000,2.900000e+02,...,2.900000e+02,290.000000,2.900000e+02,290.000000,290.000000,290.000000,2.900000e+02,290.000000,0.0,0.0
mean,205138.858586,10.887897,0.894138,5.769724,0.729379,0.000032,0.003793,1027.870069,12.859862,1.514595e+09,...,1.514560e+09,1.120690,1.514597e+09,8.054517,260.348276,7.736207,1.514603e+09,1.118517,NaN,NaN
std,87.028926,4.075870,0.210251,4.352041,0.061360,0.000118,0.013314,5.074000,4.505526,7.775896e+04,...,7.788298e+04,0.326329,7.750724e+04,1.750064,10.533490,2.079878,6.265194e+04,0.504287,NaN,NaN
min,204990.000000,1.230000,0.010000,-3.010000,0.590000,0.000000,0.000000,1017.140000,1.230000,1.514476e+09,...,1.514437e+09,1.000000,1.514473e+09,3.780000,252.000000,2.880000,1.514484e+09,0.100000,NaN,NaN
25%,205064.000000,8.217500,0.890000,3.262500,0.690000,0.000000,0.000000,1027.230000,11.515000,1.514548e+09,...,1.514524e+09,1.000000,1.514560e+09,8.270000,253.000000,5.370000,1.514578e+09,0.620000,NaN,NaN
50%,205138.000000,11.595000,1.000000,5.050000,0.710000,0.000000,0.000000,1027.685000,13.235000,1.514561e+09,...,1.514524e+09,1.000000,1.514560e+09,8.560000,253.000000,8.910000,1.514578e+09,1.250000,NaN,NaN
75%,205213.000000,13.040000,1.000000,8.480000,0.780000,0.000000,0.000000,1028.800000,16.377500,1.514655e+09,...,1.514610e+09,1.000000,1.514646e+09,9.050000,271.000000,9.000000,1.514660e+09,1.310000,NaN,NaN
max,205290.000000,18.980000,1.000000,14.840000,0.900000,0.001100,0.070000,1039.960000,19.510000,1.514766e+09,...,1.514696e+09,2.000000,1.514732e+09,9.990000,289.000000,9.370000,1.514696e+09,1.970000,NaN,NaN


In [68]:
dfwx.sample(5)

,REQUEST_ID,current.apparentTemperature,current.cloudCover,current.dewPoint,current.humidity,current.icon,current.precipIntensity,current.precipProbability,current.precipType,current.pressure,...,daily.time,daily.uvIndex,daily.uvIndexTime,daily.visibility,daily.windBearing,daily.windGust,daily.windGustTime,daily.windSpeed,0,1
0,205075,12.65,1.00,4.60,0.70,cloudy,0.0000,0.00,,1027.82,...,1.514524e+09,1.0,1.514560e+09,8.56,253.0,8.94,1.514578e+09,1.24,NaN,NaN
0,205246,1.86,0.02,-2.75,0.80,clear-night,0.0000,0.00,,1028.80,...,1.514696e+09,1.0,1.514732e+09,9.08,265.0,5.31,1.514696e+09,0.55,NaN,NaN
0,205039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,205225,13.01,1.00,12.02,0.76,cloudy,0.0000,0.00,,1018.81,...,1.514610e+09,1.0,1.514646e+09,4.30,272.0,8.91,1.514660e+09,1.83,NaN,NaN
0,205285,14.25,1.00,6.91,0.72,cloudy,0.0004,0.04,snow,1030.67,...,1.514696e+09,1.0,1.514732e+09,9.08,263.0,5.32,1.514696e+09,0.55,NaN,NaN


In [69]:
with open('311weather.csv', 'w') as fh:
    dfwx.to_csv(fh, index=False)